In [10]:
import pandas as pd
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay

## Data Preprocessing

Data is taken from the supplementary sheet of the paper - https://pmc.ncbi.nlm.nih.gov/articles/PMC5435511/

In [11]:
data_llna = pd.read_csv("LLNA.csv")
#data_human = pd.read_csv("Human.csv")

In [12]:
data_llna = data_llna.rename(columns={"DPRA\navg %\nDepletion\nLysCys":"DPRA",
                          "KeratinoSens\nEC1.5\n(ug/mL)":"KeratinoSens",
                          "hCLAT\nCD86\nCD54 \nMIT (ug/mL)":"hCLAT",
                          "MW (g/mol)":"MW",
                          "Log S mol/L":"LogS",
                          "Log VP mm Hg":"LogVP",
                          "Log P":"LogP",
                          "MP oC":"MPoC",
                          "BP oC":"BPoC",
                          "LLNA Potency GHS Category":"Category"
                         })

#data_human = data_human.rename(columns={"DPRA\navg %\nDepletion\nLysCys":"DPRA",
#                          "KeratinoSens\nEC1.5\n(ug/mL)":"KeratinoSens",
#                          "hCLAT\nCD86\nCD54 \nMIT (ug/mL)":"hCLAT",
#                          "MW (g/mol)":"MW",
#                          "Log S mol/L":"LogS",
#                          "Log VP mm Hg":"LogVP",
#                          "Log P":"LogP",
#                          "MP oC":"MPoC",
#                          "BP oC":"BPoC",
#                          "Human Potency GHS Category":"Category"
#                         })
                        

In [13]:
data_llna = data_llna.drop(columns = ["Chemical Name","CASRN","Pre/Pro-hapten?","Pre/Pro-hapten Reference","Product Class","Structure"])
#data_human = data_human.drop(columns = ["Chemical Name","CASRN","Pre/Pro-hapten?","Pre/Pro-hapten Reference","Product Class","Structure"])

In [14]:
label_map = {"NEG": 0, "1B": 1, "1A": 2}

data_llna['Category'] = data_llna['Category'].map(label_map)
#data_human['Category'] = data_human['Category'].map(label_map)

In [15]:
#checking if all data are numerics
features = ['DPRA', 'KeratinoSens', 'hCLAT', 'MW', 'LogP', 'LogS', 'LogVP', 'MPoC', 'BPoC']
for col in features:
    data_llna[col] = pd.to_numeric(data_llna[col], errors='coerce')


print(data_llna[features].isna().sum())  # Shows how many NaNs per column

DPRA            0
KeratinoSens    0
hCLAT           0
MW              0
LogP            0
LogS            0
LogVP           3
MPoC            0
BPoC            0
dtype: int64


In [16]:
data_llna[features] = data_llna[features].fillna(data_llna[features].median())

In [18]:
print(data_llna[features].isna().sum())

DPRA            0
KeratinoSens    0
hCLAT           0
MW              0
LogP            0
LogS            0
LogVP           0
MPoC            0
BPoC            0
dtype: int64


## Test Train Split

In [19]:
X_train = data_llna[data_llna['LLNA Test or Training Set'] == 'Training'][features]
y_train = data_llna[data_llna['LLNA Test or Training Set'] == 'Training']['Category']
X_test = data_llna[data_llna['LLNA Test or Training Set'] == 'Test'][features]
y_test = data_llna[data_llna['LLNA Test or Training Set'] == 'Test']['Category']

### Feature Importance

In [20]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()
rf.fit(X_train, y_train)

importances = rf.feature_importances_

# Match with feature names
for name, imp in zip(features, importances):
    print(f"{name}: {imp:.4f}")


DPRA: 0.2236
KeratinoSens: 0.1336
hCLAT: 0.1558
MW: 0.0706
LogP: 0.1187
LogS: 0.0663
LogVP: 0.0799
MPoC: 0.0902
BPoC: 0.0614


## Normalization - Standard Scaling

In [10]:
from sklearn.preprocessing import StandardScaler

In [11]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Model Architecture:



There are two ways in which we can do the prediction.


MODEL 1: Single Tier - Here, all the three categories(Weak, Strong and Non sensitized) are classified in one go. 


MODEL 2: Two Tier model - Here, fist the classification is done on the basis of sensitizer and non sensitizer. Then the classification is done for Strong and Weak.

## Model 1 : Single Tier
### Support Vector Machine

In [12]:
from sklearn.svm import SVC
clf = SVC(kernel='rbf', probability=True)
clf.fit(X_train_scaled, y_train)

SVC(probability=True)

In [13]:
from sklearn.metrics import classification_report, confusion_matrix
y_pred = clf.predict(X_test_scaled)

print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred, target_names=["Strong", "Weak", "Non-sensitizer"]))
print("\nThe predictions:",y_pred,"\n")


accuracy_svm = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy_svm)

Confusion Matrix:
 [[6 1 0]
 [3 8 1]
 [0 3 4]]

Classification Report:
                 precision    recall  f1-score   support

        Strong       0.67      0.86      0.75         7
          Weak       0.67      0.67      0.67        12
Non-sensitizer       0.80      0.57      0.67         7

      accuracy                           0.69        26
     macro avg       0.71      0.70      0.69        26
  weighted avg       0.70      0.69      0.69        26


The predictions: [2 2 1 0 2 0 1 1 0 0 1 1 2 1 1 2 0 1 0 1 0 0 1 1 0 1] 

Accuracy: 0.6923076923076923


### Random Forest Classifier

In [14]:
from sklearn.ensemble import RandomForestClassifier

In [15]:
rf = RandomForestClassifier()
rf.fit(X_train_scaled, y_train)

RandomForestClassifier()

In [16]:
y_pred = rf.predict(X_test_scaled)

In [17]:
print(y_pred)

[2 2 1 0 2 1 1 1 0 0 1 1 2 1 0 2 0 2 0 1 0 0 1 1 0 1]


In [18]:
print("Classification report\n",classification_report(y_pred, y_test))
print("\n Y predictions - ",y_pred,"\n")


Classification report
               precision    recall  f1-score   support

           0       0.71      0.56      0.62         9
           1       0.50      0.55      0.52        11
           2       0.57      0.67      0.62         6

    accuracy                           0.58        26
   macro avg       0.60      0.59      0.59        26
weighted avg       0.59      0.58      0.58        26


 Y predictions -  [2 2 1 0 2 1 1 1 0 0 1 1 2 1 0 2 0 2 0 1 0 0 1 1 0 1] 



### Logistic regression 

In [19]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression(solver="lbfgs", max_iter=1000)
logreg.fit(X_train_scaled, y_train)

y_pred = logreg.predict(X_test_scaled)


In [20]:
print("Classification report\n",classification_report(y_pred, y_test))
print("\n Y predictions - ",y_pred,"\n")

Classification report
               precision    recall  f1-score   support

           0       0.71      0.56      0.62         9
           1       0.58      0.54      0.56        13
           2       0.43      0.75      0.55         4

    accuracy                           0.58        26
   macro avg       0.58      0.61      0.58        26
weighted avg       0.60      0.58      0.58        26


 Y predictions -  [2 2 1 0 2 1 1 1 0 0 0 1 2 1 1 1 0 1 0 1 0 0 1 1 0 1] 



## Model 2 : Two Tier
### Binary Classification model(Sensitizer and non sensitizer)
#### Using Support Vector Machine


In [21]:
# TIER 1 LABEL: Sensitizer (Strong+Weak = 1) vs Non-sensitizer = 0
data_llna['tier1_label'] = data_llna['Category'].map({
    2: 1,
    1: 1,
    0: 0
})

In [22]:
X_t1_train = data_llna[data_llna['LLNA Test or Training Set'] == 'Training'][features]
y_t1_train = data_llna[data_llna['LLNA Test or Training Set'] == 'Training']['tier1_label']

X_t1_test = data_llna[data_llna['LLNA Test or Training Set'] == 'Test'][features]
y_t1_test = data_llna[data_llna['LLNA Test or Training Set'] == 'Test']['tier1_label']


In [23]:
scaler1 = StandardScaler()
X_t1_train_scaled = scaler1.fit_transform(X_t1_train)
X_t1_test_scaled = scaler1.transform(X_t1_test)


In [24]:
tier1_model = SVC(kernel='rbf', probability=True)
tier1_model.fit(X_t1_train_scaled, y_t1_train)

tier1_preds = tier1_model.predict(X_t1_test_scaled)

In [25]:
print("Confusion Matrix:\n", confusion_matrix(y_t1_test, tier1_preds))
print("\nClassification Report:\n", classification_report(y_t1_test, tier1_preds, target_names=["Sensitizer", "Non-sensitizer"]))
print("\nThe predictions:",tier1_preds,"\n")

Confusion Matrix:
 [[ 5  2]
 [ 1 18]]

Classification Report:
                 precision    recall  f1-score   support

    Sensitizer       0.83      0.71      0.77         7
Non-sensitizer       0.90      0.95      0.92        19

      accuracy                           0.88        26
     macro avg       0.87      0.83      0.85        26
  weighted avg       0.88      0.88      0.88        26


The predictions: [1 1 1 1 1 0 1 1 0 1 1 1 1 1 1 1 0 1 0 1 1 0 1 1 0 1] 



88% accuracy (Sensitizer and non Sensitizer)
### Classification between Strong and Weak
#### Using SVM as it gave the best result in our single tier model (69%)

In [26]:
# Tier 2 labels: 1A = 1 (strong), 1B = 0 (weak), NEG = NaN
data_llna['tier2_label'] = data_llna['Category'].apply(lambda x: 1 if x == 2 else (0 if x == 1 else None))

# Training set: true sensitizers only
tier2_train_df = data_llna[
    (data_llna['LLNA Test or Training Set'] == 'Training') & 
    (data_llna['tier2_label'].notna())
]

# Test set: among predicted sensitizers
test_indices = data_llna[data_llna['LLNA Test or Training Set'] == 'Test'].index
predicted_sensitizers = test_indices[tier1_preds == 1]

tier2_test_df = data_llna.loc[predicted_sensitizers]
tier2_test_df = tier2_test_df[tier2_test_df['tier2_label'].notna()]


In [27]:
X_t2_train = tier2_train_df[features]
y_t2_train = tier2_train_df['tier2_label']

X_t2_test = tier2_test_df[features]
y_t2_test = tier2_test_df['tier2_label']


In [28]:
scaler2 = StandardScaler()
X_t2_train_scaled = scaler2.fit_transform(X_t2_train)
X_t2_test_scaled = scaler2.transform(X_t2_test)

tier2_model = SVC(kernel='rbf', probability=True)
tier2_model.fit(X_t2_train_scaled, y_t2_train)

tier2_preds = tier2_model.predict(X_t2_test_scaled)


In [29]:
final_preds = []
tier2_counter = 0

for idx, pred in zip(test_indices, tier1_preds):
    if pred == 0:
        final_preds.append(0)  # Non-sensitizer (NEG)
    else:
        if idx in y_t2_test.index:
            final_preds.append(2 if tier2_preds[tier2_counter] == 1 else 1)  # 2 = 1A, 1 = 1B
            tier2_counter += 1
        else:
            final_preds.append(1)  # Fallback: assume weak (1B)


In [30]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# True labels from original test set (mapped to: NEG=0, 1B=1, 1A=2)
true_final_labels = data_llna.loc[test_indices, 'Category'].tolist()

print("Confusion Matrix:\n", confusion_matrix(true_final_labels, final_preds))

print("\nClassification Report:")
print(classification_report(true_final_labels, final_preds, target_names=["NEG", "1B", "1A"]))

print("\nOverall Accuracy:", round(accuracy_score(true_final_labels, final_preds) * 100, 2), "%")


Confusion Matrix:
 [[ 5  2  0]
 [ 1 10  1]
 [ 0  3  4]]

Classification Report:
              precision    recall  f1-score   support

         NEG       0.83      0.71      0.77         7
          1B       0.67      0.83      0.74        12
          1A       0.80      0.57      0.67         7

    accuracy                           0.73        26
   macro avg       0.77      0.71      0.73        26
weighted avg       0.75      0.73      0.73        26


Overall Accuracy: 73.08 %


In [32]:
import joblib
# Save Tier 1 model and scaler
joblib.dump(tier1_model, 'tier1_model.pkl')
joblib.dump(scaler1, 'scaler1.pkl')

# Save Tier 2 model and scaler
joblib.dump(tier2_model, 'tier2_model.pkl')
joblib.dump(scaler2, 'scaler2.pkl')

['scaler2.pkl']

### Taking input from the user 

In [9]:
import pandas as pd
import joblib

def predict_skin_sensitization(user_input=None):
    # Load saved models and scalers
    tier1_model = joblib.load('tier1_model.pkl')
    scaler1 = joblib.load('scaler1.pkl')
    tier2_model = joblib.load('tier2_model.pkl')
    scaler2 = joblib.load('scaler2.pkl')

    # Define feature order
    features = ['DPRA', 'KeratinoSens', 'hCLAT', 'MW', 'LogP', 'LogS', 'LogVP', 'MPoC', 'BPoC']

    # If no user_input passed, prompt interactively
    if user_input is None:
        print("Please enter values for the 9 input features:")
        user_input = {}
        for feature in features:
            while True:
                try:
                    value = float(input(f"Enter {feature}: "))
                    user_input[feature] = value
                    break
                except ValueError:
                    print(f"Invalid input for {feature}. Please enter a numeric value.")

    # Convert to DataFrame
    X_new = pd.DataFrame([user_input])[features]
    X_new = X_new.fillna(X_new.median())  # Safe fallback

    # --- Tier 1: Sensitizer vs Non-sensitizer ---
    X_new_scaled_t1 = scaler1.transform(X_new)
    tier1_pred = tier1_model.predict(X_new_scaled_t1)[0]

    # --- Tier 2: If sensitizer, predict Strong vs Weak ---
    if tier1_pred == 0:
        final_pred = 0  # NEG
    else:
        X_new_scaled_t2 = scaler2.transform(X_new)
        tier2_pred = tier2_model.predict(X_new_scaled_t2)[0]
        final_pred = 2 if tier2_pred == 1 else 1  # 2 = 1A, 1 = 1B

    # Decode prediction
    label_map_rev = {0: "NEG", 1: "1B", 2: "1A"}
    print("\n🧬 Predicted Skin Sensitization Class:", label_map_rev[final_pred])

    return label_map_rev[final_pred]


In [8]:
predict_skin_sensitization()

Please enter values for the 9 input features:


Enter DPRA:  46.7
Enter KeratinoSens:  5.4
Enter hCLAT:  40
Enter MW:  156.252
Enter LogP:  0.74
Enter LogS:  -1.492
Enter LogVP:  -10.064
Enter MPoC:  71.44
Enter BPoC:  122



🧬 Predicted Skin Sensitization Class: 1A


'1A'